In [29]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from dotenv import load_dotenv
from IPython.display import Markdown
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain.prompts import PromptTemplate

In [99]:
outline_prompt_string = """Context: {text}
As an expert legal analyst, review the Context which is part of an insurance policy document related to the question.
{question}

Identify Context excerpts that are of specific material importance to the question. 
For excerpts indentified, explain their specific relevancy to the question.
"""

OUTLINE_PROMPT = PromptTemplate(input_variables=["text", "question"], template=outline_prompt_string)

In [49]:
final_prompt_string = """Context: {text}
As an experienced legal analyst, use the Context and the question:
{question}

Compose a case brief using only the Context. 
Working step by step, organize the Context into a well-structured 
outline with the following sections:

Facts of Loss: a concise sumary of facts and events relevant to this claim.
Coverage Issue: [the facts of the coverage question]
Analysis: subsections
   ["Policy excerpts": excerpts from the most relevant policy subsections, 
   "Applications": explanation of how the policy excerpt impacts the question]
Takeaway: the most important policy section for the question.
TODO: optional list of knowledge gaps, or follow-up questions.
"""
FINAL_PROMPT = PromptTemplate(input_variables=["text", "question"], template=final_prompt_string)

In [50]:
from llama_index import SimpleDirectoryReader
from langchain.text_splitter import CharacterTextSplitter

sample_ho3_policy_docs = SimpleDirectoryReader(input_files=['./data//HO3_sample.pdf']).load_data()

In [90]:
docs = []

for i in range(len(sample_ho3_policy_docs)):
    doc = sample_ho3_policy_docs[i].text
    docs.append(doc)

In [91]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=2000, chunk_overlap=0, separator="\n\n")

In [92]:
policy_text = "\n\n".join(docs)

In [93]:
policy_chunks = text_splitter.split_text(policy_text)

In [94]:
len(policy_chunks)

12

In [107]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=OUTLINE_PROMPT
)

In [108]:
question = """
    The insured was away on vacation and rented the dwelling to a tenant. When the insured came home 
    he found a ceiling collapsed due to a pipe leak in the attic causing significant damage. 
    The damage was hidden from the insured, but the tenant was aware of it. Is this a covered claim?
"""

In [109]:
all_res = []

for i in range(len(policy_chunks)):
    res = llm_chain.predict(text=policy_chunks[i], question=question)
    all_res.append(res)

### GPT-3.5-turbo: Intermediate Output

In [110]:
for i in range(len(all_res)):
    display(Markdown(f"{all_res[i]}"))

Excerpt 1: "In this policy, 'you' and 'your' refer to the 'named insured' shown in the Declarations and the spouse if a resident of the same household."

Excerpt 2: "Insured location means: a. The 'residence premises'; b. The part of other premises, other structures and grounds used by you as a residence; and c. Any premises used by you in connection with a premises described in a. and b. above."

Excerpt 3: "Residence premises means: a. The one family dwelling where you reside; b. The two, three or four family dwelling where you reside in at least one of the family units; or c. That part of any other building where you reside; and which is shown as the 'residence premises' in the Declarations."

Excerpt 4: "An 'insured' does not mean a person or organization using or having custody of these animals or watercraft in the course of any 'business' or without consent of the owner."

Excerpt 5: "Under both Sections I and II, when the word an immediately precedes the word 'insured', the words an 'insured' together mean one or more 'insureds'."

Excerpt 6: "With respect to a 'motor vehicle' to which this policy applies: (a) Persons while engaged in your employ or that of any person included in a. or b. above; or (b) Other persons using the vehicle on an 'insured location' with your consent."

Excerpt 7: "Occurrence means an accident, including continuous or repeated exposure to substantially the same general harmful conditions, which results, during the policy period, in: a. 'Bodily injury'; or b. 'Property damage'."

Excerpt 8: "Property damage means physical injury to, destruction of, or loss of use of tangible property."

Excerpt 9: "Residence employee means: a. An employee of an 'insured', or an employee leased to an 'insured' by a labor leasing firm, under an agreement between an 'insured' and the labor leasing firm, whose duties are related to the maintenance or use of the 'residence premises', including household or domestic services; or b. One who performs similar duties elsewhere not related to the 'business' of an 'insured'."

Excerpt 10: "A 'residence employee' does not include a temporary employee who is furnished to an 'insured' to substitute for a permanent 'residence employee' on leave or to meet seasonal or short-term workload conditions."

Excerpt 1 is relevant because it defines who the "you" and "your" refer to in the policy, which is important in determining coverage.

Excerpts 2 and 3 are relevant because they define the "residence premises" and "insured location," which are important in determining whether the damage occurred within the covered property.

Excerpt 4 is relevant because it clarifies that the policy does not cover damages caused by someone using or having custody of the property without the owner's consent.

Excerpt 5 is relevant because it explains that when the term "an insured" is used, it refers to one or more insureds, which may include the tenant.

Excerpt 6 is relevant because it states that other persons using the vehicle on an insured location with the insured's consent may be covered.

Excerpt 7 is relevant because it defines "occurrence," which is important in determining whether the damage caused by the pipe leak is covered.

Excerpt 8 is relevant because it defines "property damage," which is the type of damage caused by the pipe leak.

Excerpt 9 is relevant because it defines "residence employee," which may be relevant in determining who is responsible for the maintenance and care of the property.

Excerpt 10 is relevant because it clarifies that a temporary employee is not considered a residence employee, which may be relevant in determining who is responsible for the maintenance and care of the property.

Excerpt 1: "Coverage A – Dwelling...We cover: a. The dwelling on the 'residence premises' shown in the Declarations, including structures attached to the dwelling..."

Excerpt 2: "Coverage B – Other Structures...We cover other structures on the 'residence premises' set apart from the dwelling by clear space. This includes structures connected to the dwelling by only a fence, utility line, or similar connection."

Excerpt 3: "Coverage C – Personal Property...We cover personal property owned or used by an 'insured' while it is anywhere in the world."

Excerpt 4: "We do not cover: h. Property rented or held for rental to others off the 'residence premises'."

Excerpt 5: "We do not cover: i. 'Business' data, including such data stored in: (1) Books of account, drawings or other paper records; or (2) Computers and related equipment."

The relevant excerpts are Excerpt 1, Excerpt 2, and Excerpt 4. 

Excerpt 1 states that the insurance policy covers the dwelling on the "residence premises" shown in the Declarations, including structures attached to the dwelling. This means that if the damage occurred to the dwelling itself, it would likely be covered.

Excerpt 2 states that the insurance policy also covers other structures on the "residence premises" set apart from the dwelling. If the damage occurred to a structure connected to the dwelling by a fence, utility line, or similar connection, it would likely be covered.

Excerpt 4 states that the insurance policy does not cover property rented or held for rental to others off the "residence premises." If the damage occurred to a property that was rented to someone off the "residence premises," it would not be covered.

Based on the information provided, it is unclear whether the damage occurred to the dwelling itself or to a separate structure. If the damage occurred to the dwelling or a structure on the "residence premises," it would likely be covered. However, if the damage occurred to a property rented to someone off the "residence premises," it would not be covered.

Excerpt 1: "If a loss covered under Section I makes that part of the 'residence premises' rented to others or held for rental by you not fit to live in, we cover the fair rental value of such premises less any expenses that do not continue while it is not fit to live in."

Excerpt 2: "We cover trees, shrubs, plants or lawns, on the 'residence premises', for loss caused by the following Perils Insured Against: [...] Theft."

Excerpt 3: "We will pay up to $500 for: [...] Loss resulting from theft or unauthorized use of an electronic fund transfer card or access device used for deposit, withdrawal or transfer of funds, issued to or registered in an 'insured's' name."

Excerpt 1 is relevant because it states that if a loss makes the rented part of the residence premises not fit to live in, the insurance policy covers the fair rental value of such premises. In this case, the collapsed ceiling due to a pipe leak would likely make the rented part of the dwelling not fit to live in, so the claim may be covered.

Excerpt 2 is relevant because it states that the insurance policy covers loss to trees, shrubs, plants, or lawns caused by theft. If the tenant was aware of the hidden damage and intentionally caused or allowed the damage to occur, it could be considered theft, and the claim may be covered.

Excerpt 3 is relevant because it states that the insurance policy covers loss resulting from theft or unauthorized use of an electronic fund transfer card or access device. If the tenant used the insured's credit card or access device without authorization to cause the damage, it could be considered theft, and the claim may be covered.

Excerpt 8a states that collapse is defined as an abrupt falling down or caving in of a building or any part of a building that renders it uninhabitable. In this case, the ceiling collapse would be considered a collapse under this definition.

Excerpt 8b states that the insurance policy covers direct physical loss to covered property involving collapse if it was caused by certain perils, including the Perils Insured Against named under Coverage C. If the pipe leak in the attic caused the collapse of the ceiling, it would be considered a covered claim under this provision.

Excerpt 8c states that loss to certain property, such as an awning or foundation, is not included under the coverage for collapse unless it is a direct result of the collapse of a building or part of a building. Since the damage in this case is to the ceiling, which is part of the building, it would be included under the coverage for collapse.

Based on these excerpts, it can be concluded that the damage caused by the ceiling collapse due to the pipe leak would likely be considered a covered claim under the insurance policy.

Excerpt 1: "Shut off the water supply and drain all systems and appliances of water. However, if the building is protected by an automatic fire protective sprinkler system, you must use reasonable care to continue the water supply and maintain heat in the building for coverage to apply."

Relevancy: This excerpt states that if the building is protected by an automatic fire protective sprinkler system, the insured must use reasonable care to maintain the water supply and heat in the building for coverage to apply. This suggests that if the insured failed to maintain the water supply and heat in the building, the claim may not be covered.

Excerpt 2: "(5) Mold, fungus or wet rot. However, we do insure for loss caused by mold, fungus or wet rot that is hidden within the walls or ceilings or beneath the floors or above the ceilings of a structure if such loss results from the accidental discharge or overflow of water or steam from within: (a) A plumbing, heating, air conditioning or automatic fire protective sprinkler system, or a household appliance, on the 'residence premises'; or (b) A storm drain, or water, steam or sewer pipes, off the 'residence premises'."

Relevancy: This excerpt states that the insurance policy covers loss caused by mold, fungus, or wet rot that is hidden within the structure if it results from the accidental discharge or overflow of water or steam from certain sources. If the pipe leak in the attic resulted in the mold, fungus, or wet rot that caused the ceiling collapse, it may be covered under this provision.

Excerpt 3: "(8) Vandalism Or Malicious Mischief. (9) Theft. a. This peril includes attempted theft and loss of property from a known place when it is likely that the property has been stolen. b. This peril does not include loss caused by theft: (1) Committed by an 'insured'; (3) From that part of a 'residence premises' rented by an 'insured' to someone other than another 'insured'."

Relevancy: This excerpt states that theft is covered under the insurance policy, but it does not include theft committed by an insured or theft from a part of the residence premises rented by an insured to someone other than another insured. If the tenant was responsible for the theft of the insured's property, it may not be covered under this provision.

...

Excerpt (b) is of specific material importance to the question. It states that if the amount of insurance on the damaged building is less than 80% of the full replacement cost, the insurance company will pay the greater of the actual cash value of the damaged part of the building or the proportion of the cost to repair or replace the damaged part of the building, based on the ratio of the insurance amount to 80% of the replacement cost.

In this case, if the insured intentionally concealed or misrepresented the material fact that the tenant was aware of the damage, it could be considered fraud. If the insurance company determines that fraud has occurred, they may deny coverage for the claim.

Excerpt 1: "Coverage E – Personal Liability
If a claim is made or a suit is brought against an "insured" for damages because of "bodily injury" or "property damage" caused by an "occurrence" to which this coverage applies, we will:
1. Pay up to our limit of liability for the damages for which an "insured" is legally liable."

Excerpt 2: "SECTION II – EXCLUSIONS
A. "Motor Vehicle Liability"
1. Coverages E and F do not apply to any "motor vehicle liability" if, at the time and place of an "occurrence", the involved "motor vehicle":
a. Is registered for use on public roads or property;
b. Is not registered for use on public roads or property, but such registration is required by a law, or regulation issued by a government agency, for it to be used at the place of the "occurrence"; or
c. Is being:
(1) Operated in, or practicing for, any prearranged or organized race, speed contest or other competition;
(2) Rented to others;
(3) Used to carry persons or cargo for a charge; or
(4) Used for any "business" purpose except for a motorized golf cart while on a golfing facility."

Excerpt 1 is relevant to the question because it states that the insurance policy will pay for damages for which the insured is legally liable. This implies that if the insured is found to be legally liable for the damages caused by the pipe leak, the claim may be covered.

Excerpt 2 is relevant to the question because it states that the insurance policy does not cover any "motor vehicle liability" if certain conditions are met. This implies that if the damage caused by the pipe leak is considered "motor vehicle liability" and the conditions listed in Excerpt 2 are met, the claim may not be covered.

Excerpt 1: "Coverages E and F do not apply to the following:...4. 'Property damage' to property rented to, occupied or used by or in the care of an 'insured'. This exclusion does not apply to 'property damage' caused by fire, smoke or explosion."

Excerpt 2: "The insured was away on vacation and rented the dwelling to a tenant. When the insured came home he found a ceiling collapsed due to a pipe leak in the attic causing significant damage."

Excerpt 1 is relevant to the question because it states that coverage for property damage to property rented to an insured is excluded, unless the damage is caused by fire, smoke, or explosion. This indicates that the claim may not be covered under the policy.

Excerpt 2 provides the specific scenario in question, where the insured rented the dwelling to a tenant and discovered damage upon returning home. This excerpt helps to establish the context and relevance of the question.

Excerpt 1: "Coverage F does not apply to 'bodily injury'...to any person, other than a 'residence employee' of an 'insured', regularly residing on any part of the 'insured location'."

Excerpt 2: "We will pay, at replacement cost, up to $1,000 per 'occurrence' for 'property damage' to property of others caused by an 'insured'."

Excerpt 3: "We will not pay for 'property damage'...to property owned by or rented to a tenant of an 'insured' or a resident in your household."

Excerpt 4: "This insurance applies separately to each 'insured'. This condition will not increase our limit of liability for any one 'occurrence'."

Excerpt 5: "In case of an 'occurrence', you or another 'insured' will perform the following duties that apply...Promptly forward to us every notice, demand, summons or other process relating to the 'occurrence'."

Excerpt 1 is relevant because it states that Coverage F, which includes coverage for medical payments to others, does not apply to any person other than a residence employee regularly residing on the insured location. This means that if the tenant is injured, they would not be covered under Coverage F.

Excerpt 2 is relevant because it states that the insurance policy will pay for property damage caused by an insured, up to $1,000 per occurrence. This suggests that if the damage to the ceiling is considered property damage and is caused by the insured, there may be coverage for the claim.

Excerpt 3 is relevant because it states that the insurance policy will not pay for property damage to property owned by or rented to a tenant of the insured. This suggests that if the damage to the ceiling is considered property damage and is caused by the insured, there may not be coverage for the claim.

Excerpt 4 is relevant because it states that the insurance applies separately to each insured and does not increase the limit of liability for any one occurrence. This means that even if the insured and the tenant are both considered insureds under the policy, the limit of liability for property damage remains the same.

Excerpt 5 is relevant because it states that in case of an occurrence, the insured or another insured must promptly forward any notice or process relating to the occurrence to the insurance company. This suggests that the insured has a duty to notify the insurance company of the claim and provide any relevant information or documentation.

Excerpt 5 is of specific material importance to the question. It states that for damage to property of others, the insured must submit a sworn statement of loss within 60 days after the loss. This excerpt is relevant because it establishes a requirement for the insured to report the damage within a specific timeframe.

Excerpt 6 is also of specific material importance to the question. It states that no insured shall voluntarily make payment, assume obligation, or incur expense other than for first aid to others at the time of bodily injury, except at their own cost. This excerpt is relevant because it suggests that the insured should not have taken any action to repair the damage without notifying the insurance company first.

Based on these excerpts, it can be inferred that the insured's claim may not be covered if they did not report the damage within the specified timeframe and if they took action to repair the damage without the insurance company's knowledge or consent.

In [104]:
outline_text = "\n".join(all_res)

In [111]:
final_chain = LLMChain(
    llm=ChatOpenAI(model="gpt-3.5-turbo-16k-0613", temperature=0),
    prompt=FINAL_PROMPT
)

In [112]:
final_res = final_chain.predict(text=outline_text, question=question)

## GPT-3.5-turbo: Final Answer

In [113]:
Markdown(f"{final_res}")

Facts of Loss:
- The insured was away on vacation and rented the dwelling to a tenant.
- Upon returning home, the insured discovered a collapsed ceiling due to a pipe leak in the attic, causing significant damage.
- The damage was hidden from the insured, but the tenant was aware of it.

Coverage Issue:
- Is this a covered claim under the insurance policy?

Analysis:
- Policy excerpts:
  - Excerpt 1: "Coverage E – Personal Liability...If a claim is made or a suit is brought against an 'insured' for damages because of 'bodily injury' or 'property damage' caused by an 'occurrence' to which this coverage applies, we will pay up to our limit of liability for the damages for which an 'insured' is legally liable."
  - Excerpt 3: "We will not pay for 'property damage'...to property owned by or rented to a tenant of an 'insured' or a resident in your household."
- Applications:
  - Excerpt 1 indicates that the insurance policy covers damages caused by an occurrence, such as the pipe leak, for which the insured is legally liable.
  - However, Excerpt 3 states that the insurance policy will not pay for property damage to property rented to a tenant of the insured.
- Takeaway:
  - Based on the policy excerpts, it appears that the insurance policy does not cover property damage to property rented to a tenant of the insured.

TODO: None.

In [114]:
llm = ChatOpenAI(model="gpt-4", temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=OUTLINE_PROMPT
)

In [115]:
all_res = []

for i in range(len(policy_chunks)):
    res = llm_chain.predict(text=policy_chunks[i], question=question)
    all_res.append(res)

### GPT-4: Intermediate Output

In [116]:
for i in range(len(all_res)):
    display(Markdown(f"{all_res[i]}"))

Excerpt 1: "We will provide the insurance described in this policy in return for the premium and compliance with all applicable provisions of this policy."

Relevancy: This excerpt establishes the general agreement between the insurer and the insured. It implies that the insurer will cover damages as long as the insured complies with the policy's provisions. 

Excerpt 2: "8."Occurrence" means an accident, including continuous or repeated exposure to substantially the same general harmful conditions, which results, during the policy period, in: a."Bodily injury"; or b."Property damage"."

Relevancy: This excerpt defines an "occurrence" as an accident that results in property damage during the policy period. The pipe leak and subsequent ceiling collapse could be considered an "occurrence" as it resulted in property damage.

Excerpt 3: "9."Property damage" means physical injury to, destruction of, or loss of use of tangible property."

Relevancy: This excerpt defines "property damage" as physical injury to or destruction of tangible property. The damage caused by the pipe leak and ceiling collapse falls under this definition.

Excerpt 4: "h.Any part of a premises occasionally rented to an "insured" for other than "business" use."

Relevancy: This excerpt indicates that the policy covers premises that are occasionally rented for non-business use. Since the insured rented out the dwelling while on vacation, this provision could apply.

However, the policy does not explicitly state whether it covers damages that were known by the tenant but not reported to the insured. This could be a point of contention in determining whether the claim is covered.

Excerpt 1: "We cover: a. The dwelling on the "residence premises" shown in the Declarations, including structures attached to the dwelling; and b. Materials and supplies located on or next to the "residence premises" used to construct, alter or repair the dwelling or other structures on the "residence premises"."
Relevancy: This excerpt is relevant because it establishes that the insurance policy covers the dwelling on the residence premises, which would include the ceiling that collapsed due to the pipe leak.

Excerpt 2: "We do not cover: a. Land, including land on which the other structures are located; b. Other structures rented or held for rental to any person not a tenant of the dwelling, unless used solely as a private garage; c. Other structures from which any "business" is conducted; or d. Other structures used to store "business" property."
Relevancy: This excerpt is relevant because it specifies what the policy does not cover. In this case, the dwelling was rented out to a tenant, which could potentially affect the coverage. However, the policy does not explicitly state that it does not cover dwellings rented out to tenants.

Excerpt 3: "We cover personal property owned or used by an "insured" while it is anywhere in the world. After a loss and at your request, we will cover personal property owned by: a. Others while the property is on the part of the "residence premises" occupied by an "insured"; or b. A guest or a "residence employee", while the property is in any residence occupied by an "insured"."
Relevancy: This excerpt is relevant because it establishes that the policy covers personal property owned or used by the insured. However, it does not explicitly state that it covers damage to the dwelling caused by a tenant.

Excerpt 4: "Property Not Covered: We do not cover: a. Articles separately described and specifically insured, regardless of the limit for which they are insured, in this or other insurance; b. Animals, birds or fish; c. "Motor vehicles"."
Relevancy: This excerpt is relevant because it specifies what property is not covered by the policy. However, it does not explicitly state that it does not cover damage caused by a tenant.

In conclusion, while the policy covers the dwelling and personal property owned or used by the insured, it does not explicitly state whether it covers damage caused by a tenant. Therefore, it is unclear whether this would be a covered claim. The insured may need to consult with their insurance provider for clarification.

Excerpt 1: "2. Fair Rental Value If a loss covered under Section I makes that part of the "residence premises" rented to others or held for rental by you not fit to live in, we cover the fair rental value of such premises less any expenses that do not continue while it is not fit to live in. Payment will be for the shortest time required to repair or replace such premises."

Relevancy: This excerpt is relevant because it outlines the coverage for rental properties. The insured's property was rented out at the time of the damage, so this clause could potentially apply. It states that if a covered loss makes the rented premises unfit to live in, the insurance will cover the fair rental value of the premises during the repair or replacement period.

Excerpt 2: "2. Reasonable Repairs a.We will pay the reasonable cost incurred by you for the necessary measures taken solely to protect covered property that is damaged by a Peril Insured Against from further damage."

Relevancy: This excerpt is relevant because it states that the insurance will cover the cost of necessary measures taken to protect the property from further damage. This could potentially apply to the situation described in the question, as the pipe leak caused damage to the property.

However, it's important to note that the policy does not explicitly state whether damage that was known by the tenant but not reported to the insured would be covered. This could potentially be a point of contention in determining whether this claim is covered.

Excerpt 1: "We will pay up to $1,000 for your share of loss assessment charged during the policy period against you, as owner or tenant of the "residence premises", by a corporation or association of property owners. The assessment must be made as a result of direct loss to property, owned by all members collectively, of the type that would be covered by this policy if owned by you, caused by a Peril Insured Against under Coverage A, other than: (1)Earthquake; or (2)Land shock waves or tremors before, during or after a volcanic eruption."

Relevancy: This excerpt is relevant because it outlines the conditions under which the insurance company will pay for a loss assessment charged against the policyholder. In this case, the policyholder is the owner of the residence premises and the loss is a result of direct damage to the property, which could potentially be covered under the policy.

Excerpt 2: "Collapse means an abrupt falling down or caving in of a building or any part of a building with the result that the building or part of the building cannot be occupied for its current intended purpose."

Relevancy: This excerpt is relevant because it defines what constitutes a "collapse" under the policy. The situation described in the question involves a ceiling collapse, which would fall under this definition.

Excerpt 3: "We insure for direct physical loss to covered property involving collapse of a building or any part of a building if the collapse was caused by one or more of the following: (1)The Perils Insured Against named under Coverage C; (2)Decay that is hidden from view, unless the presence of such decay is known to an "insured" prior to collapse; (3)Insect or vermin damage that is hidden from view, unless the presence of such damage is known to an "insured" prior to collapse;"

Relevancy: This excerpt is relevant because it outlines the conditions under which the insurance company will cover a collapse. In this case, the collapse was caused by a pipe leak, which could be considered a "Peril Insured Against" under Coverage C. Additionally, the damage was hidden from the insured, which could potentially meet the conditions for coverage.

Excerpt 4: "We do not insure, however, for loss: a.Excluded under Section I – Exclusions; b.Involving collapse, except as provided in E.8. Collapse under Section I – Property Coverages; or c.Caused by: (1)Freezing of a plumbing, heating, air conditioning or automatic fire protective sprinkler system or of a household appliance, or by discharge, leakage or overflow from within the system or appliance caused by freezing."

Relevancy: This excerpt is relevant because it outlines the exclusions to the policy's coverage. In this case, the damage was not caused by freezing, so this exclusion would not apply. However, the policyholder would need to review Section I – Exclusions to ensure that the damage is not excluded under any other provisions.

Excerpt 1: "However, if the building is protected by an automatic fire protective sprinkler system, you must use reasonable care to continue the water supply and maintain heat in the building for coverage to apply."

Relevancy: This excerpt is relevant because it establishes that the insured is expected to maintain the property, including the water supply and heating system, to ensure coverage. If the pipe leak was due to negligence in maintaining these systems, the claim might not be covered.

Excerpt 2: "For purposes of this provision, a plumbing system or household appliance does not include a sump, sump pump or related equipment or a roof drain, gutter, downspout or similar fixtures or equipment."

Relevancy: This excerpt is relevant because it clarifies what is considered part of the plumbing system. If the pipe that leaked is part of the plumbing system as defined in the policy, the damage caused by the leak could be covered.

Excerpt 3: "We do insure for loss caused by mold, fungus or wet rot that is hidden within the walls or ceilings or beneath the floors or above the ceilings of a structure if such loss results from the accidental discharge or overflow of water or steam from within: (a) A plumbing, heating, air conditioning or automatic fire protective sprinkler system, or a household appliance, on the "residence premises"

Relevancy: This excerpt is relevant because it establishes that the policy does cover damage caused by hidden water leaks from the plumbing system. If the pipe leak was hidden and caused the ceiling to collapse, this could be a covered claim.

Excerpt 4: "We do not cover loss to the system or appliance from which this water or steam escaped."

Relevancy: This excerpt is relevant because it clarifies that while the damage caused by the leak may be covered, the cost to repair or replace the leaking pipe itself may not be covered.

Excerpt 5: "Under 2.b. and c. above, any ensuing loss to property described in Coverages A and B not precluded by any other provision in this policy is covered."

Relevancy: This excerpt is relevant because it suggests that the ensuing damage to the property (in this case, the collapsed ceiling) could be covered, provided it is not excluded by any other provision in the policy. 

Excerpt 6: "This peril does not include loss caused by theft: (2) In or to a dwelling under construction, or of materials and supplies for use in the construction until the dwelling is finished and occupied;"

Relevancy: This excerpt is relevant because it establishes that the policy does not cover loss in a dwelling under construction or unoccupied. If the insured's dwelling was considered unoccupied during the vacation, this could affect the coverage.

...

Excerpt 1: "If the building is rebuilt at a new premises, the cost described in (2) above is limited to the cost which would have been incurred if the building had been built at the original premises."

Relevancy: This excerpt is relevant because it outlines the policy's coverage for rebuilding costs. If the insured decides to rebuild the damaged dwelling, the policy will cover the cost as if it were rebuilt at the original location. 

Excerpt 2: "We will pay no more than the actual cash value of the damage until actual repair or replacement is complete. Once actual repair or replacement is complete, we will settle the loss as noted in 2.a. and b. above."

Relevancy: This excerpt is relevant because it explains how the insurance company will pay for the damage. The company will pay the actual cash value of the damage until the repair or replacement is complete. 

Excerpt 3: "You may disregard the replacement cost loss settlement provisions and make claim under this policy for loss to buildings on an actual cash value basis."

Relevancy: This excerpt is relevant because it gives the insured the option to claim the loss on an actual cash value basis, which might be relevant depending on the extent of the damage and the cost of repair or replacement.

Excerpt 4: "No action can be brought against us unless there has been full compliance with all of the terms under Section I of this policy and the action is started within two years after the date of loss."

Relevancy: This excerpt is relevant because it sets a time limit for the insured to bring a claim against the insurance company. The insured must comply with all the terms of the policy and start the action within two years after the date of loss.

Excerpt 5: "We provide coverage to no "insureds" under this policy if, whether before or after a loss, an "insured" has: 1. Intentionally concealed or misrepresented any material fact or circumstance;"

Relevancy: This excerpt is relevant because it states that the insurance company will not provide coverage if the insured has intentionally concealed or misrepresented any material fact or circumstance. In this case, the tenant was aware of the damage but the insured was not. If it can be proven that the insured was aware of the damage and did not report it, this could potentially affect the claim.

Excerpt 1: "If the Declarations show a loss payee for certain listed insured personal property, the definition of "insured" is changed to include that loss payee with respect to that property."

Relevancy: This excerpt is relevant because it explains who is considered an "insured" under the policy. If the tenant is listed as a loss payee, they would be considered an "insured" and the policy may cover the damage. 

Excerpt 2: "If a claim is made or a suit is brought against an "insured" for damages because of "bodily injury" or "property damage" caused by an "occurrence" to which this coverage applies, we will: 1.Pay up to our limit of liability for the damages for which an "insured" is legally liable."

Relevancy: This excerpt is relevant because it outlines the insurer's obligation to pay for damages caused by an "occurrence". The pipe leak could be considered an "occurrence" and the insurer may be obligated to pay for the damages. 

Excerpt 3: "We may investigate and settle any claim or suit that we decide is appropriate. Our duty to settle or defend ends when our limit of liability for the "occurrence" has been exhausted by payment of a judgment or settlement."

Relevancy: This excerpt is relevant because it explains the insurer's right to investigate and settle any claim. This could impact the insured's claim regarding the pipe leak and the resulting damage. 

Excerpt 4: "We will pay the necessary medical expenses that are incurred or medically ascertained within three years from the date of an accident causing "bodily injury". 

Relevancy: This excerpt is not directly relevant to the question as it pertains to medical expenses and bodily injury, not property damage. However, it does provide context for the types of expenses the policy covers. 

Excerpt 5: "Coverages E and F do not apply to any "motor vehicle liability" if, at the time and place of an "occurrence", the involved "motor vehicle": a.Is registered for use on public roads or property;"

Relevancy: This excerpt is not relevant to the question as it pertains to motor vehicle liability, not property damage caused by a pipe leak.

Excerpt 1: "Coverages E and F do not apply to the following: 1. Expected Or Intended Injury "Bodily injury" or "property damage" which is expected or intended by an "insured" even if the resulting "bodily injury" or "property damage": a.Is of a different kind, quality or degree than initially expected or intended; or b.Is sustained by a different person, entity, real or personal property, than initially expected or intended."

Relevancy: This excerpt is relevant because it states that the policy does not cover damage that was expected or intended by the insured. In the scenario, the tenant was aware of the damage, but it's unclear whether the insured was aware or expected the damage. If it can be proven that the insured was aware of the potential for damage and did nothing to prevent it, this clause could potentially be used to deny the claim.

Excerpt 2: "F. Coverage E – Personal Liability Coverage E does not apply to: 3."Property damage" to property rented to, occupied or used by or in the care of an "insured". This exclusion does not apply to "property damage" caused by fire, smoke or explosion;"

Relevancy: This excerpt is relevant because it states that the policy does not cover property damage to property that is rented to, occupied, used by, or in the care of the insured. However, there is an exception for damage caused by fire, smoke, or explosion. In the scenario, the property was rented to a tenant and the damage was caused by a pipe leak, not by fire, smoke, or explosion. Therefore, this clause could potentially be used to deny the claim.

Excerpt 3: "2. "Business" a."Bodily injury" or "property damage" arising out of or in connection with a "business" conducted from an "insured location" or engaged in by an "insured", whether or not the "business" is owned or operated by an "in-sured" or employs an "insured". This Exclusion E.2. applies but is not lim- ited to an act or omission, regardless of its nature or circumstance, involving a service or duty rendered, promised, owed, or im- plied to be provided because of the nature of the "business"."

Relevancy: This excerpt is relevant because it states that the policy does not cover property damage arising out of or in connection with a business conducted from an insured location. If the insured was renting out the property as a business, this clause could potentially be used to deny the claim. However, more information would be needed to determine if this is the case.

Excerpt 1: "Coverage F does not apply to "bodily injury": 1. To a "residence employee" if the "bodily injury": a. Occurs off the "insured location"; and b. Does not arise out of or in the course of the "residence employee's" employment by an "insured";"

Relevancy: This excerpt is relevant because it outlines the conditions under which the insurance policy does not cover bodily injury. However, in the context of the question, it does not directly address the issue of property damage caused by a pipe leak.

Excerpt 2: "C. Damage To Property Of Others 1. We will pay, at replacement cost, up to $1,000 per "occurrence" for "property damage" to property of others caused by an "insured". 2. We will not pay for "property damage": a. To the extent of any amount recoverable under Section I; b. Caused intentionally by an "insured" who is 13 years of age or older; c. To property owned by an "insured"; d. To property owned by or rented to a tenant of an "insured" or a resident in your household;"

Relevancy: This excerpt is directly relevant to the question as it outlines the conditions under which the insurance policy covers property damage. Specifically, it states that the policy does not cover damage to property owned by or rented to a tenant of the insured. In the context of the question, this suggests that the damage caused by the pipe leak may not be covered by the insurance policy because the dwelling was rented to a tenant at the time of the incident.

Excerpt 3: "C. Duties After "Occurrence" In case of an "occurrence", you or another "insured" will perform the following duties that apply. We have no duty to provide coverage under this policy if your failure to comply with the following duties is prejudicial to us. You will help us by seeing that these duties are performed: 1. Give written notice to us or our agent as soon as is practical, which sets forth: a. The identity of the policy and the "named insured" shown in the Declarations; b. Reasonably available information on the time, place and circumstances of the "occurrence"; and c. Names and addresses of any claimants and witnesses;"

Relevancy: This excerpt is relevant because it outlines the insured's duties after an occurrence, including giving written notice to the insurance company as soon as practical. In the context of the question, this suggests that the insured may have a duty to report the pipe leak and the resulting damage to the insurance company.

Excerpt 1: "This policy applies only to "bodily injury" or "property damage" which occurs during the policy period."
Relevancy: This excerpt is relevant because it establishes the time frame for coverage. If the property damage occurred during the policy period, it may be covered.

Excerpt 2: "We do not provide coverage to an "insured" who, whether before or after a loss, has: 1. Intentionally concealed or misrepresented any material fact or circumstance; 2. Engaged in fraudulent conduct; or 3. Made false statements; relating to this insurance."
Relevancy: This excerpt is relevant because it addresses the issue of concealment or misrepresentation. If the tenant intentionally concealed the damage from the insured, it could potentially impact the claim.

Excerpt 3: "You may cancel this policy at any time by returning it to us or by letting us know in writing of the date cancellation is to take effect."
Relevancy: This excerpt is relevant because it establishes the insured's right to cancel the policy. If the insured had cancelled the policy while on vacation, the damage may not be covered.

Excerpt 4: "If there has been a material misrepresentation of fact which if known to us would have caused us not to issue the policy; or (2) If the risk has changed substantially since the policy was issued."
Relevancy: This excerpt is relevant because it addresses the issue of material misrepresentation and change in risk. If the insured did not inform the insurance company about renting the dwelling to a tenant, it could be considered a material misrepresentation or a substantial change in risk, which could impact the claim.

Excerpt 5: "We insure the legal representative of the deceased but only with respect to the premises and property of the deceased covered under the policy at the time of death; and 2."Insured" includes: a. An "insured" who is a member of your household at the time of your death, but only while a resident of the "residence premises"; and b. With respect to your property, the person having proper temporary custody of the property until appointment and qualification of a legal representative."
Relevancy: This excerpt is relevant because it defines who is considered an "insured" under the policy. If the tenant is considered an "insured", the claim could potentially be covered.

In [117]:
Markdown(f"{all_res[3]}")

Excerpt 1: "We will pay up to $1,000 for your share of loss assessment charged during the policy period against you, as owner or tenant of the "residence premises", by a corporation or association of property owners. The assessment must be made as a result of direct loss to property, owned by all members collectively, of the type that would be covered by this policy if owned by you, caused by a Peril Insured Against under Coverage A, other than: (1)Earthquake; or (2)Land shock waves or tremors before, during or after a volcanic eruption."

Relevancy: This excerpt is relevant because it outlines the conditions under which the insurance company will pay for a loss assessment charged against the policyholder. In this case, the policyholder is the owner of the residence premises and the loss is a result of direct damage to the property, which could potentially be covered under the policy.

Excerpt 2: "Collapse means an abrupt falling down or caving in of a building or any part of a building with the result that the building or part of the building cannot be occupied for its current intended purpose."

Relevancy: This excerpt is relevant because it defines what constitutes a "collapse" under the policy. The situation described in the question involves a ceiling collapse, which would fall under this definition.

Excerpt 3: "We insure for direct physical loss to covered property involving collapse of a building or any part of a building if the collapse was caused by one or more of the following: (1)The Perils Insured Against named under Coverage C; (2)Decay that is hidden from view, unless the presence of such decay is known to an "insured" prior to collapse; (3)Insect or vermin damage that is hidden from view, unless the presence of such damage is known to an "insured" prior to collapse;"

Relevancy: This excerpt is relevant because it outlines the conditions under which the insurance company will cover a collapse. In this case, the collapse was caused by a pipe leak, which could be considered a "Peril Insured Against" under Coverage C. Additionally, the damage was hidden from the insured, which could potentially meet the conditions for coverage.

Excerpt 4: "We do not insure, however, for loss: a.Excluded under Section I – Exclusions; b.Involving collapse, except as provided in E.8. Collapse under Section I – Property Coverages; or c.Caused by: (1)Freezing of a plumbing, heating, air conditioning or automatic fire protective sprinkler system or of a household appliance, or by discharge, leakage or overflow from within the system or appliance caused by freezing."

Relevancy: This excerpt is relevant because it outlines the exclusions to the policy's coverage. In this case, the damage was not caused by freezing, so this exclusion would not apply. However, the policyholder would need to review Section I – Exclusions to ensure that the damage is not excluded under any other provisions.

In [118]:
final_chain = LLMChain(
    llm=llm,
    prompt=FINAL_PROMPT
)

outline_text = "\n".join(all_res)

In [119]:
final_res = final_chain.predict(text=outline_text, question=question)

## GPT-4:

In [120]:
Markdown(f"{final_res}")

Facts of Loss:
The insured was away on vacation and rented the dwelling to a tenant. During this time, a pipe in the attic leaked, causing the ceiling to collapse and resulting in significant damage. The tenant was aware of the damage, but the insured was not informed until he returned home.

Coverage Issue:
The main issue is whether the insurance policy covers the damage caused by the pipe leak and subsequent ceiling collapse, given that the tenant was aware of the damage but did not inform the insured.

Analysis:
Policy Excerpts:
1. "We will provide the insurance described in this policy in return for the premium and compliance with all applicable provisions of this policy."
2. "8."Occurrence" means an accident, including continuous or repeated exposure to substantially the same general harmful conditions, which results, during the policy period, in: a."Bodily injury"; or b."Property damage"."
3. "9."Property damage" means physical injury to, destruction of, or loss of use of tangible property."
4. "h.Any part of a premises occasionally rented to an "insured" for other than "business" use."

Applications:
The policy provides coverage for property damage resulting from an "occurrence", which could include the pipe leak and ceiling collapse. However, the policy also requires compliance with all applicable provisions, which could potentially include the tenant's responsibility to report the damage. The policy does cover premises occasionally rented for non-business use, which could apply to this situation. However, the policy does not explicitly state whether it covers damages that were known by the tenant but not reported to the insured.

Takeaway:
The most important policy section for this question is the provision that covers premises occasionally rented for non-business use. This provision could potentially apply to the insured's situation. However, the policy does not explicitly state whether it covers damages that were known by the tenant but not reported to the insured. This could be a point of contention in determining whether the claim is covered.

Knowledge Gaps:
1. Does the policy have any provisions regarding the responsibilities of tenants?
2. Does the policy have any exclusions that could potentially apply to this situation?
3. How does the policy define "compliance with all applicable provisions"? Does this include the tenant's responsibility to report damage?

In [121]:
question = """
    The insured rented the home through AirB&B for several months. When the tenants moved out
    he found they had converted the dwelling into a marijuana grow operation. This resulted in water damage and an
    electrical fire. Is there coverage for the dwelling dweling damage, and do any named perils apply to the personal property damage?
"""

In [122]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=OUTLINE_PROMPT
)

all_res = []
for i in range(len(policy_chunks)):
    res = llm_chain.predict(text=policy_chunks[i], question=question)
    all_res.append(res)

In [125]:
Markdown(f"{all_res[3]}")

Excerpt 8a: "Collapse means an abrupt falling down or caving in of a building or any part of a building with the result that the building or part of the building cannot be occupied for its current intended purpose."
Relevancy: This excerpt defines what constitutes a collapse, which is important in determining whether the damage caused by the marijuana grow operation would be covered under the policy.

Excerpt 8b: "We insure for direct physical loss to covered property involving collapse of a building or any part of a building if the collapse was caused by one or more of the following: (1) The Perils Insured Against named under Coverage C..."
Relevancy: This excerpt states that coverage for collapse is provided if it is caused by one of the named perils under Coverage C. It is relevant in determining whether the damage caused by the marijuana grow operation would be covered.

Excerpt 11a: "You may use up to 10% of the limit of liability that applies to Coverage A for the increased costs you incur due to the enforcement of any ordinance or law which requires or regulates: (1) The construction, demolition, remodeling, renovation or repair of that part of a covered building or other structure damaged by a Peril Insured Against..."
Relevancy: This excerpt explains that the policy provides coverage for increased costs due to the enforcement of any ordinance or law related to the construction, demolition, remodeling, renovation, or repair of a covered building damaged by a named peril. It is relevant in determining whether the costs of repairing the water damage and electrical fire caused by the marijuana grow operation would be covered.

Excerpt 12: "We will pay up to $5,000 for grave markers, including mausoleums, on or away from the 'residence premises' for loss caused by a Peril Insured Against under Coverage C."
Relevancy: This excerpt specifies coverage for grave markers, but it is not directly relevant to the question about coverage for dwelling damage or personal property damage caused by the marijuana grow operation.

In [123]:
outline_text = "\n".join(all_res)

In [124]:
final_chain = LLMChain(
    llm=ChatOpenAI(model="gpt-3.5-turbo-16k-0613", temperature=0),
    prompt=FINAL_PROMPT
)
final_res = final_chain.predict(text=outline_text[:15000], question=question)

In [126]:
Markdown(f"{final_res}")

Facts of Loss:
- The insured rented the home through AirB&B for several months.
- The tenants converted the dwelling into a marijuana grow operation.
- This resulted in water damage and an electrical fire.

Coverage Issue:
- Is there coverage for the dwelling damage caused by the water damage and electrical fire?
- Do any named perils apply to the personal property damage?

Analysis:
- Policy excerpts:
  - Excerpt 1: "Coverage A – Dwelling"
    - Coverage for the dwelling on the "residence premises" shown in the Declarations.
    - Covers structures attached to the dwelling and materials and supplies used to construct, alter, or repair the dwelling or other structures on the "residence premises."
  - Excerpt 2: "Coverage B – Other Structures"
    - Coverage for other structures on the "residence premises" set apart from the dwelling by clear space.
    - Structures connected to the dwelling by only a fence, utility line, or similar connection are covered.
  - Excerpt 3: "Coverage C – Personal Property"
    - Coverage for personal property owned or used by an "insured" anywhere in the world.
    - Personal property owned by others while on the part of the "residence premises" occupied by an "insured" or by a guest or "residence employee" in any residence occupied by an "insured" can be covered.
  - Excerpt 4: "Special Limits Of Liability"
    - Specific limits for different categories of personal property.
    - Limits for money, securities, watercraft, jewelry, firearms, silverware, property used for "business" purposes, and electronic apparatus and accessories.

- Applications:
  - Excerpt 1: Coverage A – Dwelling
    - This coverage may apply to the dwelling damage caused by the water damage and electrical fire.
  - Excerpt 2: Coverage B – Other Structures
    - This coverage may apply to any other structures on the "residence premises" that were damaged.
  - Excerpt 3: Coverage C – Personal Property
    - This coverage may apply to the personal property damage caused by the tenants' conversion of the dwelling into a marijuana grow operation.
  - Excerpt 4: Special Limits Of Liability
    - The limits specified in this excerpt may be important in determining the coverage for the personal property damage caused by the marijuana grow operation.

Takeaway:
- The most important policy section for the question is "Coverage A – Dwelling" as it specifically addresses coverage for the dwelling damage caused by the water damage and electrical fire.

TODO: None.

In [127]:
llm = ChatOpenAI(model="gpt-4", temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=OUTLINE_PROMPT
)

all_res = []
for i in range(len(policy_chunks)):
    res = llm_chain.predict(text=policy_chunks[i], question=question)
    all_res.append(res)


In [128]:
Markdown(f"{all_res[3]}")

Excerpt 1: "We will pay up to $1,000 for your share of loss assessment charged during the policy period against you, as owner or tenant of the "residence premises", by a corporation or association of property owners. The assessment must be made as a result of direct loss to property, owned by all members collectively, of the type that would be covered by this policy if owned by you, caused by a Peril Insured Against under Coverage A, other than: (1)Earthquake; or (2)Land shock waves or tremors before, during or after a volcanic eruption."

Relevance: This excerpt is relevant because it outlines the policy's coverage for loss assessment charged against the policyholder due to direct loss to property. However, the policy does not specify whether damage caused by illegal activities (such as a marijuana grow operation) is considered a "Peril Insured Against under Coverage A". 

Excerpt 2: "We insure for direct physical loss to covered property involving collapse of a building or any part of a building if the collapse was caused by one or more of the following: (1)The Perils Insured Against named under Coverage C; (2)Decay that is hidden from view, unless the presence of such decay is known to an "insured" prior to collapse; (3)Insect or vermin damage that is hidden from view, unless the presence of such damage is known to an "insured" prior to collapse; (4)Weight of contents, equipment, animals or people; (5)Weight of rain which collects on a roof; or (6)Use of defective material or methods in construction, remodeling or renovation if the collapse occurs during the course of the construction, remodeling or renovation."

Relevance: This excerpt is relevant because it outlines the policy's coverage for direct physical loss to covered property involving collapse of a building. However, it does not specify whether damage caused by illegal activities (such as a marijuana grow operation) is considered a "Peril Insured Against under Coverage C". 

Excerpt 3: "We do not cover assessments charged against you or a corporation or association of property owners by any governmental body."

Relevance: This excerpt is relevant because it specifies that the policy does not cover assessments charged against the policyholder by any governmental body. This could potentially apply if the policyholder is fined or otherwise financially penalized by a governmental body due to the illegal marijuana grow operation. 

Excerpt 4: "We do not insure, however, for loss: a.Excluded under Section I – Exclusions; b.Involving collapse, except as provided in E.8. Collapse under Section I – Property Coverages; or c.Caused by: (1)Freezing of a plumbing, heating, air conditioning or automatic fire protective sprinkler system or of a household appliance, or by discharge, leakage or overflow from within the system or appliance caused by freezing."

Relevance: This excerpt is relevant because it outlines some of the exclusions to the policy's coverage. However, it does not specify whether damage caused by illegal activities (such as a marijuana grow operation) is excluded from coverage.

In [129]:
outline_text = "\n".join(all_res)

In [130]:
final_chain = LLMChain(
    llm=llm,
    prompt=FINAL_PROMPT
)
final_res = final_chain.predict(text=outline_text, question=question)

In [131]:
Markdown(f"{final_res}")

Facts of Loss:
The insured rented out their home through AirB&B for several months. The tenants converted the dwelling into a marijuana grow operation, which resulted in water damage and an electrical fire. 

Coverage Issue:
The question is whether the insurance policy covers the dwelling damage and whether any named perils apply to the personal property damage caused by the tenants' marijuana grow operation.

Analysis:
Policy Excerpts:
1. "DEFINITIONS A.In this policy, "you" and "your" refer to the "named insured" shown in the Declarations and the spouse if a resident of the same household. "We", "us" and "our" refer to the Company providing this insurance."
2. "3."Business" means: a.A trade, profession or occupation engaged in on a full-time, part-time or occasional basis; or b.Any other activity engaged in for money or other compensation, except the following: (1)One or more activities, not described in(2) through (4) below, for which no "insured" receives more than $2,000 in total compensation for the 12 months before the beginning of the policy period;"
3. "6."Insured location" means: a.The "residence premises"; b.The part of other premises, other structures and grounds used by you as a residence; and (1)Which is shown in the Declarations; or (2)Which is acquired by you during the policy period for your use as a residence;"
4. "8."Occurrence" means an accident, including continuous or repeated exposure to substantially the same general harmful conditions, which results, during the policy period, in: a."Bodily injury"; or b."Property damage"."
5. "9."Property damage" means physical injury to, destruction of, or loss of use of tangible property."
6. "h.Any part of a premises occasionally rented to an "insured" for other than "business" use."

Applications:
The policy defines the insured as the person who rented out their home through AirB&B. The insured's activity of renting out their home could potentially be considered a business activity, which may affect the coverage. The home that was rented out and subsequently damaged would fall under the definition of an "insured location". The water damage and electrical fire caused by the tenants' marijuana grow operation could potentially be considered an "occurrence" that results in property damage. The policy also specifies that a premises occasionally rented for non-business use is considered an "insured location". This could potentially apply to the insured's situation, depending on whether the rental through AirB&B is considered a business activity.

Takeaway:
The most important policy section for this question is the definition of "occurrence" and "property damage". If the water damage and electrical fire can be considered an "occurrence" that results in "property damage", then the policy may cover the damages. However, the determination of coverage may also depend on whether the rental through AirB&B is considered a business activity.

Knowledge Gaps:
The policy does not explicitly state whether damage caused by illegal activities, such as a marijuana grow operation, is covered. Further information would be needed to determine whether such activities are covered or excluded under the policy.

* Wrap up the above logic into a function

In [146]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from llama_index import SimpleDirectoryReader

def get_policy_chunks(file_path: str):
    policy_docs = SimpleDirectoryReader(input_files=[file_path]).load_data()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=2000, chunk_overlap=0, separator="\n\n")

    docs = []
    for i in range(len(policy_docs)):
        doc = policy_docs[i].text
        docs.append(doc)

    policy_text = "\n\n".join(docs)
    policy_chunks = text_splitter.split_text(policy_text)
    return policy_chunks

def generate_policy_analysis(file_path: str, question: str) -> str:
    """
    Generate a policy analysis based on the given file path and question.

    This function reads the policy document from the given file path, splits it into chunks, and then uses
    a language model to generate a policy analysis based on the given question. The analysis is returned as a string.

    Args:
        file_path (str): The path to the policy document.
        question (str): The question to base the analysis on.

    Returns:
        str: The generated policy analysis.
    """
    
    policy_chunks = get_policy_chunks(file_path=file_path)

    outline_prompt_string = """Context: {text}
    As an expert legal analyst, review the Context which is part of an insurance policy document related to the question.
    {question}

    Identify Context excerpts that are of material importance to the question. 
    For excerpts indentified, explain their relevancy to the question.
    """

    OUTLINE_PROMPT = PromptTemplate(input_variables=["text", "question"], template=outline_prompt_string)

    final_prompt_string = """Context: {text}
    As an experienced legal analyst, use the Context and the question:
    {question}

    Compose a case brief using only the Context. 
    Working step by step, organize the Context into a well-structured 
    outline with the following sections:

    Facts of Loss: a concise sumary of facts and events relevant to this claim.
    Coverage Issue: [the facts of the coverage question]
    Analysis: subsections
       ["Policy excerpts": excerpts from the most relevant policy subsections, 
       "Applications": explanation of how the policy excerpt impacts the question]
    Takeaway: the most important policy section for the question.
    TODO: optional list of knowledge gaps, or follow-up questions.
    """
    FINAL_PROMPT = PromptTemplate(input_variables=["text", "question"], template=final_prompt_string)

    llm = ChatOpenAI(model="gpt-4", temperature=0)

    policy_excerpt_chain = LLMChain(
        llm=llm,
        prompt=OUTLINE_PROMPT
    )

    all_res = []
    for i in range(len(policy_chunks)):
        res = policy_excerpt_chain.predict(text=policy_chunks[i], question=question)
        all_res.append(res)

    outline_text = "\n\n".join(all_res)

    final_chain = LLMChain(
        llm=llm,
        prompt=FINAL_PROMPT
    )

    final_res = final_chain.predict(text=outline_text, question=question)
    
    return final_res

In [19]:
file_path = './data//HO3_sample.pdf'

question = """
    Lack of ventalation in the attic caused exsessive moisture build up resulting in mold damage. Engineer report confirmed the cause is 
    due to faulty workmanship. Is this a covered claim?
"""

In [134]:
from src_index.token_catcher import Usage

In [148]:
usage = Usage()
result = generate_policy_analysis(file_path, question)
usage.total_tokens()

29974

In [145]:
29699/1000*0.03

0.89097

In [149]:
Markdown(f"{result}")

Facts of Loss: The insured has filed a claim for mold damage in the attic of their property. The mold damage was caused by excessive moisture build-up due to a lack of ventilation in the attic. An engineer's report confirmed that the cause of the moisture build-up was faulty workmanship.

Coverage Issue: The question is whether the mold damage caused by faulty workmanship and lack of ventilation in the attic is a covered claim under the insured's policy.

Analysis: 
Policy Excerpts:
1. "We will provide the insurance described in this policy in return for the premium and compliance with all applicable provisions of this policy."
2. "8."Occurrence" means an accident, including continuous or repeated exposure to substantially the same general harmful conditions, which results, during the policy period, in: a."Bodily injury"; or b."Property damage"."
3. "9."Property damage" means physical injury to, destruction of, or loss of use of tangible property."
4. "SECTION I – PROPERTY COVERAGES A. Coverage A – Dwelling 1.We cover: a.The dwelling on the "residence premises" shown in the Declarations, including structures attached to the dwelling; and b.Materials and supplies located on or next to the "residence premises" used to construct, alter or repair the dwelling or other structures on the "residence premises"."

Applications:
1. The policy provides insurance coverage in return for the premium and compliance with all applicable provisions of the policy. This indicates that the insured must comply with all the policy's provisions for the claim to be covered.
2. The policy defines an "occurrence" as an accident, including continuous or repeated exposure to substantially the same general harmful conditions, resulting in property damage. If the mold damage can be considered as "property damage" resulting from "continuous or repeated exposure to substantially the same general harmful conditions", then it might be covered under the policy.
3. The policy defines "property damage" as physical injury to, destruction of, or loss of use of tangible property. If the mold damage can be considered as "physical injury to, destruction of, or loss of use of tangible property", then it might be covered under the policy.
4. The policy covers the dwelling on the residence premises, including structures attached to the dwelling. The mold damage caused by excessive moisture build-up due to faulty workmanship could potentially fall under this coverage, as it is damage to the dwelling itself.

Takeaway: The most important policy sections for this question are the definitions of "occurrence" and "property damage", and the coverage for the dwelling. These sections could potentially provide coverage for the mold damage claim, depending on the specific provisions of the policy.

TODO: 
1. Review the full policy document to determine if there are any specific exclusions or conditions related to mold damage or damage resulting from faulty workmanship.
2. Consult with the insurance company for clarification on whether the mold damage caused by faulty workmanship and lack of ventilation in the attic is a covered claim.
3. Review the engineer's report to determine if there are any additional factors that could impact the coverage of the claim.

In [16]:
from llama_index import SimpleDirectoryReader, ServiceContext, ListIndex, StorageContext, VectorStoreIndex
from llama_index.node_parser import SimpleNodeParser

In [2]:
policy_docs = SimpleDirectoryReader(input_files=['./data//HO3_sample.pdf']).load_data()

In [6]:
parser = SimpleNodeParser.from_defaults(
    chunk_size=3000,
)

In [14]:
storage_context = StorageContext.from_defaults()

In [7]:
nodes = parser.get_nodes_from_documents(policy_docs)

In [15]:
storage_context.docstore.add_documents(nodes)

In [8]:
len(policy_docs), len(nodes)

(22, 22)

In [9]:
policy_docs[0].text

'HOMEOWNERS\nHO 00 03 10 00\nHO 00 03 10 00 Copyright, Insurance Services Office, Inc., 1999 Page 1 of 22HOMEOWNERS 3 – SPECIAL FORM\nAGREEMENT\nWe will provide the insurance described in this policy\nin return for the premium and compliance with allapplicable provisions of this policy.\nDEFINITIONS\nA.In this policy, "you" and "your" refer to the "named\ninsured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this in-surance.\nB.In addition, certain words and phrases are definedas follows:\n1."Aircraft Liability", "Hovercraft Liability", "Motor\nVehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the\nfollowing:\na.Liability for "bodily injury" or "property dam-age" arising out of the:\n(1)Ownership of such vehicle or craft by an"insured";\n(2)Maintenance, occupancy, operation,use, loading or unloading of such vehi-cle or craft by any person;\n(3)Entrustment of such v

In [10]:
nodes[0].text

'HOMEOWNERS\nHO 00 03 10 00\nHO 00 03 10 00 Copyright, Insurance Services Office, Inc., 1999 Page 1 of 22HOMEOWNERS 3 – SPECIAL FORM\nAGREEMENT\nWe will provide the insurance described in this policy\nin return for the premium and compliance with allapplicable provisions of this policy.\nDEFINITIONS\nA.In this policy, "you" and "your" refer to the "named\ninsured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this in-surance.\nB.In addition, certain words and phrases are definedas follows:\n1."Aircraft Liability", "Hovercraft Liability", "Motor\nVehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the\nfollowing:\na.Liability for "bodily injury" or "property dam-age" arising out of the:\n(1)Ownership of such vehicle or craft by an"insured";\n(2)Maintenance, occupancy, operation,use, loading or unloading of such vehi-cle or craft by any person;\n(3)Entrustment of such v

In [12]:
policy_index = ListIndex(nodes)

In [17]:
list_index = ListIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

In [43]:
list_query_engine = list_index.as_query_engine(
    response_mode="accumulate", use_async=True
)

In [44]:
import nest_asyncio

nest_asyncio.apply()

In [53]:
query = f"List the policy clauses that are relevant to the following question. Include complete and accurate policy quotes for each relevant clause. \nQUESTION: {question}"

In [54]:
print(query)

List the policy clauses that are relevant to the following question. Include complete an daccurate policy quotes for each relevant clause. 
QUESTION: 
    Lack of ventalation in the attic caused exsessive moisture build up resulting in mold damage. Engineer report confirmed the cause is 
    due to faulty workmanship. Is this a covered claim?



In [55]:
response = list_query_engine.query(query)

C:\Users\pdoub\.pyenv\pyenv-win\versions\3.10.1\lib\copy.py:230: RuntimeWarning: coroutine 'LLMPredictor.apredict' was never awaited
  for key, value in x.items():
C:\Users\pdoub\.pyenv\pyenv-win\versions\3.10.1\lib\copy.py:230: RuntimeWarning: coroutine 'run_async_tasks.<locals>._gather' was never awaited
  for key, value in x.items():


In [56]:
print(f"{response}")

Response 1: The relevant policy clause for the question is the "Covered Perils" clause. The policy quote for this clause is not provided in the given context.
---------------------
Response 2: The relevant policy clauses for the question are the ones related to "Property damage" and "Occurrence". 

The policy defines "Property damage" as physical injury to, destruction of, or loss of use of tangible property. 

The policy defines "Occurrence" as an accident, including continuous or repeated exposure to substantially the same general harmful conditions, which results, during the policy period, in bodily injury or property damage. 

Based on these policy clauses, the claim for mold damage caused by faulty workmanship would likely be covered, as long as the damage occurred during the policy period and meets the definition of an "Occurrence" and "Property damage".
---------------------
Response 3: The relevant policy clause for the question is the "Coverage A - Dwelling" clause. The policy

In [34]:
print(f"{response}")

Response 1: The relevant policy clauses that can help determine if this claim is covered are the ones related to property damage and exclusions.
---------------------
Response 2: The relevant policy clauses that can help determine if this claim is covered are the definitions of "occurrence" and "property damage".
---------------------
Response 3: The relevant policy clauses that can help determine if this claim is covered are the ones related to Coverage A - Dwelling and Coverage B - Other Structures.
---------------------
Response 4: The policy clauses that are relevant to the question are not mentioned in the given context information.
---------------------
Response 5: The relevant policy clauses for the question are Section I - Property Coverages and Coverage D - Loss Of Use.
---------------------
Response 6: The relevant policy clauses for the question are not mentioned in the given context information.
---------------------
Response 7: The relevant policy clause for the question i